# Functions as first class citizens

In [1]:
def mul(a, b):
    return a*b

mul(2, 3)

6

In [2]:
mul = lambda a, b: a*b

mul(2, 3)

6

# Lambda is another way of defining a function

# Higher Order Functions

# Functions as arguments to other functions

In [3]:
mul(mul(2, 3), 3)

18

In [4]:
def transform_and_add(func, a, b):
    return func(a) + func(b)

transform_and_add(lambda x: x**2, 1, 2)

5

# Why would I want something like this?

# A Familiar Pattern

In [5]:
def square_and_add(a, b):
    return (a**2 + b**2)

def cube_and_add(a, b):
    return (a**3 + b**3)

def quad_and_add(a, b):
    return (a**4 + b**4)

print(square_and_add(1, 2))
print(cube_and_add(1, 2))
print(quad_and_add(1, 2))

5
9
17


In [6]:
square = lambda x: x**2
cube = lambda x: x**3
quad = lambda x: x**4
print(square_and_add(1, 2) == transform_and_add(square, 1, 2))
print(cube_and_add(1, 2) == transform_and_add(cube, 1, 2))
print(quad_and_add(1, 2) == transform_and_add(quad, 1, 2))

True
True
True


In [7]:
def square_and_add(a, b):
    return (a**2 + b**2)

def cube_and_mul(a, b):
    return ((a**3) * (b**3))

def quad_and_div(a, b):
    return ((a**4) / (b**4))

print(square_and_add(1, 2))
print(cube_and_mul(1, 2))
print(quad_and_div(1, 2))

5
8
0.0625


In [8]:
def transform_and_reduce(func_transform, func_reduce, a, b):
    return func_reduce(func_transform(a), func_transform(b))

print(square_and_add(1, 2) == transform_and_reduce(square, lambda x, y: x+y, 1, 2))
print(cube_and_mul(1, 2) == transform_and_reduce(cube, lambda x, y: x*y, 1, 2))
print(quad_and_div(1, 2) == transform_and_reduce(quad, lambda x, y: x/y, 1, 2))

True
True
True


In [9]:
import operator

print(square_and_add(1, 2) == transform_and_reduce(square, operator.add, 1, 2))
print(cube_and_mul(1, 2) == transform_and_reduce(cube, operator.mul, 1, 2))
print(quad_and_div(1, 2) == transform_and_reduce(quad, operator.truediv, 1, 2))

True
True
True


# Lets do some maths

# Number of transform functions = m
# Number of reduce functions = n


# Number of functions in the first workflow = m\*n

# Number of functions in the second workflow = m + n

# Write small, re-useable function

In [10]:
print(square_and_add(1, 2) == transform_and_reduce(lambda x: x**2, lambda x, y: x+y, 1, 2))
print(cube_and_mul(1, 2) == transform_and_reduce(lambda x: x**3, lambda x, y: x*y, 1, 2))
print(quad_and_div(1, 2) == transform_and_reduce(lambda x: x**4, lambda x, y: x/y, 1, 2))

True
True
True


# Function returns function as result

In [11]:
from time import time
def timer(func):
    def inner(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print("Time take = {time}".format(time = time() - t))
    return inner

def echo_func(input):
    print(input)
    
timed_echo = timer(echo_func)

timed_echo(1000000)

1000000
Time take = 0.00010395050048828125


# Partial Functions

In [12]:
def logger(level, message):
    print("{level}: {message}".format(level = level, message = message))

def debug(message):
    return logger("debug", message)

def info(messgae):
    return logger("info", message)

debug("Error 404")

debug: Error 404


In [13]:
from functools import partial

debug = partial(logger, "debug")
info = partial(logger, "info")

debug("Error 404")

debug: Error 404


# debug("Error 404") = partial(logger, "debug")("Error 404")

In [14]:
partial(logger, "debug")("Error 404")

debug: Error 404


# Currying

# f(a, b, c) => g(a)(b)(c)

In [15]:
def transform_and_add(func_transform, a, b):
    return func_transform(a) + func_transform(b)

def curry_transform_and_add(func_transform):
    def apply(a, b):
        return func_transform(a) + func_transform(b)
    
    return apply

In [16]:
print(transform_and_add(cube, 1, 2) == curry_transform_and_add(cube)(1, 2))

True


# Currying gets you specialized functions from more general functions

# Map, Reduce, Filter 

In [17]:
input_list = [1, 2, 3, 4]
squared_list = map(lambda x: x**2, input_list)
print(type(squared_list))
print(next(squared_list))
print(next(squared_list))

<class 'map'>
1
4


In [18]:
from functools import reduce
sum_list = reduce(operator.add, input_list)
print(sum_list)

10


In [19]:
sum_squared_list = reduce(operator.add, 
                          map(lambda x: x**2, input_list))
print(sum_squared_list)

30


In [20]:
even_list = list(
                filter(lambda x: x%2==0, input_list))
sum_even_list = reduce(operator.add, even_list)
print(sum_even_list)

6


In [21]:
print(reduce(operator.add,
            (map(lambda x: x**2, 
                filter(lambda x: x%2==0, input_list)))))

20


# itertools — Functions creating iterators for efficient looping

In [28]:
from itertools import accumulate
acc = accumulate(input_list, operator.add)
print(input_list)
print(type(acc))
print(next(acc))
print(next(acc))
print(next(acc))

[1, 2, 3, 4]
<class 'itertools.accumulate'>
1
3
6


# Recursion

In [29]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

# Sadly, no tail recursion

# Comprehension

In [34]:
print(input_list)
collection = list()
is_even = lambda x: x%2==0
for data in input_list:
    if(is_even(data)):
        collection.append(data)
    else:
        collection.append(data*2)
print(collection)

[1, 2, 3, 4]
[2, 2, 6, 4]


In [36]:
collection = [data if is_even(data) else data*2 
              for data in input_list]
print(collection)

[2, 2, 6, 4]


# Generators

In [37]:
collection = (data if is_even(data) else data*2 
              for data in input_list)
print(collection)

<generator object <genexpr> at 0x7ff7d02a5f78>
